# Feature Engineering, Encodage et Sélection de Features

Dans ce notebook, nous allons explorer différentes techniques de sélection et d'analyse de features, notamment :
- L'importance des features via Random Forest
- La sélection de features avec SelectKBest
- L'analyse en composantes principales (PCA)
- Les corrélations entre features

In [ ]:
# Imports nécessaires
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.datasets import fetch_california_housing
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.ensemble import RandomForestRegressor

## 1. Chargement et Préparation des Données

In [ ]:
# Chargement du dataset
data = fetch_california_housing(as_frame=True)
df = data.frame.copy()

print("Aperçu des données:")
display(df.head())
print("\nInformations sur le dataset:")
df.info()

In [ ]:
# Standardisation des features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(df.drop("MedHouseVal", axis=1))
y = df["MedHouseVal"]

print("Shape des données après préparation:")
print("X:", X_scaled.shape)
print("y:", y.shape)

## 2. Importance des Features via Random Forest

In [ ]:
# Entraînement du Random Forest pour l'importance des features
rf = RandomForestRegressor(n_estimators=100, random_state=42)
rf.fit(X_scaled, y)
importances = rf.feature_importances_

# Création d'une série pandas pour faciliter la visualisation
features = df.drop("MedHouseVal", axis=1).columns
feat_importances = pd.Series(importances, index=features).sort_values(ascending=False)

# Visualisation
plt.figure(figsize=(10,6))
sns.barplot(x=feat_importances.values, y=feat_importances.index)
plt.title("Importance des Features (Random Forest)")
plt.xlabel("Importance Score")
plt.tight_layout()
plt.show()

print("\nImportance des features (en pourcentage):")
for feat, imp in feat_importances.items():
    print(f"{feat}: {imp*100:.2f}%")

## 3. Sélection de Features avec SelectKBest

In [ ]:
# Application de SelectKBest avec f_regression
selector = SelectKBest(score_func=f_regression, k=5)
X_kbest = selector.fit_transform(X_scaled, y)
selected_features = features[selector.get_support()]

print("Top 5 features sélectionnées avec SelectKBest:")
scores = pd.Series(selector.scores_, index=features).sort_values(ascending=False)
for feat, score in scores.head().items():
    print(f"{feat}: {score:.2f}")

## 4. Analyse en Composantes Principales (PCA)

In [ ]:
# Application de PCA
pca = PCA()
X_pca = pca.fit_transform(X_scaled)

# Visualisation de la variance expliquée
plt.figure(figsize=(10,6))
plt.plot(range(1, len(pca.explained_variance_ratio_) + 1), 
         np.cumsum(pca.explained_variance_ratio_), 'bo-')
plt.xlabel('Nombre de composantes')
plt.ylabel('Variance expliquée cumulée')
plt.title('Variance expliquée cumulée vs. Nombre de composantes')
plt.grid(True)
plt.show()

print("\nVariance expliquée par composante:")
for i, var in enumerate(pca.explained_variance_ratio_):
    print(f"PC{i+1}: {var*100:.2f}%")

## 5. Analyse des Corrélations

In [ ]:
# Matrice de corrélation
plt.figure(figsize=(12,8))
sns.heatmap(df.corr(), annot=True, cmap='coolwarm', fmt='.2f')
plt.title("Matrice de Corrélation des Features")
plt.tight_layout()
plt.show()

# Identification des paires de features fortement corrélées
corr_matrix = df.corr().abs()
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
strong_corr = [(col1, col2, corr_matrix.loc[col1, col2]) 
               for col1 in corr_matrix.columns 
               for col2 in corr_matrix.columns 
               if corr_matrix.loc[col1, col2] > 0.5 and col1 < col2]

print("\nPaires de features fortement corrélées (>0.5):")
for col1, col2, corr in strong_corr:
    print(f"{col1} - {col2}: {corr:.2f}")